In [255]:
# for modeling PM policies of manufacturing systems
import numpy as np
import pandas as pd

import itertools
import random
import math
import time

import matplotlib.pyplot as plt

import simpy

In [2]:
class Problem_Solution:
    def __init__(self, pair, reliability=None, relia_params=None):
        self.problem_name, self.solution_name = pair
        self.pair = pair
        
        if reliability: # reliability is defined here
            self.reliability = reliability
            self.relia_params = relia_params
        else: # reliability is defined elsewhere
            # lookup reliability distribution
            pass
        
    def get_ttf(self):
        return self.reliability(*self.relia_params)

In [314]:
class Item:
    def __init__(self, reliability=None, env=None, sim=None, repairman=None, verbose=False):
        # get reliability for problem-solution pairs
        #p_s_pairs = itertools.product(problems, solutions)
        
        #self.problem_solution_pairs = {(pair):Problem_Solution(pair) for pair in p_s_pairs}
        
        self.verbose = verbose
        
        self.reliability = reliability
        t0_problem, t0_solution = random.choice(list(self.reliability.keys()))
        
        self.failed = False
        
        self.uptime = 0
        
        if env: # simulation environment
            self.sim = sim
            self.env = env
            self.time_of_next_failure = self.env.now + self.get_time_to_next_failure(t0_problem, t0_solution)
            self.process = self.env.process(self.working())
            self.env.process(self.breakdown())
    
    def get_time_to_first_failure(self):
        initial_p_s = random.choice(list(self.reliability.keys()))
    
    def get_time_to_next_failure(self, problem, solution):
        # sample the lifetime distributions for each failure mode
        # given a problem-solution to find the next failure
        ttf, mode = 99999, None
        lifetimes = self.reliability[(problem, solution)]
        for failure in lifetimes.keys():
            failure_dist, failure_params = lifetimes[failure]
            failure_ttf = failure_dist(failure_params)
            
            if failure_ttf < ttf:
                ttf = failure_ttf
                mode = failure
        
        self.next_failure_mode = mode
        self.next_ttf = math.floor(ttf[0])
        return self.next_ttf
    
    def get_random_repair_mode(self, problem):
        keys = list(self.reliability.keys())
        repair_modes = [k[1] for k in keys if problem in k]
        
        return random.choice(repair_modes)
        
    def working(self):
        while True:
            try:
                yield self.env.timeout(1)
                self.uptime += 1
            
            except simpy.Interrupt:
                #print('interrupted at {}'.format(self.env.now))
                repair = self.get_random_repair_mode(self.next_failure_mode)
                
                if repair == 'replace':
                    ttr = 5
                else:
                    ttr = 2
                
                yield self.env.timeout(ttr)
                if self.verbose: print('Repair: {} at {}\n'.format(repair, self.env.now))
                
                self.current_maint_job[0] += [repair, ttr]
                
                #repair_log = [[self.env.now, 'repair', repair, ttr]]
                #repair_log = pd.DataFrame(repair_log, columns=self.sim.df_cols)
                #self.sim.maintenance_data = self.sim.maintenance_data.append(repair_log, ignore_index=True)
                
                self.current_maint_job = pd.DataFrame(self.current_maint_job, columns=self.sim.df_cols)
                self.sim.maintenance_data = self.sim.maintenance_data.append(self.current_maint_job, ignore_index=True)
                
                self.time_of_next_failure = self.env.now + self.get_time_to_next_failure(self.next_failure_mode, repair)
                
    def breakdown(self):
        while True:
            yield self.env.timeout(1)
            if self.env.now == self.time_of_next_failure:
                #print(self.time_of_next_failure)
                self.failed = True
                if self.verbose: print('Failure: {} at {}'.format(self.next_failure_mode, self.env.now))
                
                self.current_maint_job = [[self.env.now, self.next_failure_mode, self.next_ttf]]
                
                #failure_log = [[self.env.now, 'failure', self.next_failure_mode, self.next_ttf]]
                #failure_log = pd.DataFrame(failure_log, columns=self.sim.df_cols)
                #self.sim.maintenance_data = self.sim.maintenance_data.append(failure_log, ignore_index=True)
                
                self.process.interrupt()
    
    def sample_reliability_dict(self):
        samp = '''
    hydraulic_reliability = {('low pressure', 'tighten'):{'low pressure':(np.random.normal, (15,)),
                                                          'leak':        (np.random.normal, (10,))},
                             ('low pressure', 'replace'):{'low pressure':(np.random.normal, (20,)),
                                                          'leak':        (np.random.normal, (25,))},
                             ('leak', 'replace'):        {'leak':        (np.random.normal, (20,))}}
    '''
        print('Reliability distribution dictionary should be of the form:')
        print(samp)
                

In [315]:
class Simulation:
    def __init__(self, relia):
        self.relia = relia
        
        self.df_cols = ['time', 'failure type', 'ttf', 'repair type', 'ttr']
        self.maintenance_data = pd.DataFrame(columns=self.df_cols)
        
    def simulate(self, verbose=False):
        self.env = simpy.Environment()
        
        self.items = []
        for r in self.relia:
            self.items += [Item(r, env=self.env, sim=self, verbose=verbose)]
            
        print('Simulation started.\n')
        start_time = time.time()
        self.env.run(until=100)
        print('runtime: {:.4f}s'.format(time.time()-start_time))        

In [321]:
# problem-solution reliability dictionary
hydraulic = {('low pressure', 'tighten'):{'low pressure':(np.random.normal, (15,)),
                                          'leak':        (np.random.normal, (10,))},
             ('low pressure', 'replace'):{'low pressure':(np.random.normal, (20,)),
                                          'leak':        (np.random.normal, (25,))},
             ('leak', 'replace'):        {'leak':        (np.random.normal, (20,)),
                                          'low pressure':(np.random.normal, (20,))}}

hydraulic = {('low pressure', 'tighten'):{'low pressure':(np.random.exponential, (9,)),
                                          'leak':        (np.random.exponential, (10,))},
             ('low pressure', 'replace'):{'low pressure':(np.random.exponential, (10,)),
                                          'leak':        (np.random.exponential, (11,))},
             ('leak', 'replace'):        {'low pressure':(np.random.exponential, (11,)),
                                          'leak':        (np.random.exponential, (12,))}}

s = Simulation([hydraulic])
s.simulate()

Simulation started.

runtime: 0.0560s


In [322]:
s.maintenance_data

,time,failure type,ttf,repair type,ttr
0,1,leak,1,replace,5
1,11,low pressure,5,tighten,2
2,13,leak,0,replace,5
3,31,leak,13,replace,5
4,40,leak,4,replace,5
5,55,low pressure,10,tighten,2
6,60,leak,3,replace,5
7,71,low pressure,6,replace,5
8,82,leak,6,replace,5
9,87,low pressure,0,replace,5


In [294]:
s.maintenance_data[s.maintenance_data['type']=='failure']

,time,type,name,ttf/ttr
0,8,failure,low pressure,8
2,12,failure,low pressure,1
4,22,failure,leak,7
6,35,failure,leak,10
8,38,failure,leak,0
10,46,failure,low pressure,5
12,52,failure,low pressure,3
14,57,failure,low pressure,2
16,71,failure,leak,11
18,75,failure,low pressure,1


In [179]:
s.items[0].uptime

87

In [87]:
hydraulic = Item(problems, solutions, hydraulic_reliability)

In [95]:
hydraulic.get_random_repair_mode('low pressure')

'tighten'

In [21]:
for k in hydraulic_reliability[('low pressure', 'tighten')].keys(): print(k)

low pressure
leak


In [58]:
print(a)


hydraulic_reliability = {('low pressure', 'tighten'):{'low pressure':(np.random.normal, (15,)),
                                                      'leak':        (np.random.normal, (10,))},
                         ('low pressure', 'replace'):{'low pressure':(np.random.normal, (20,)),
                                                      'leak':        (np.random.normal, (25,))},
                         ('leak', 'replace'):        {'leak':        (np.random.normal, (20,))}}



In [251]:
# problem-solution reliability dictionary
hydraulic = {('low pressure', 'tighten'):{'low pressure':(np.random.normal, (15,)),
                                          'leak':        (np.random.normal, (10,))},
             ('low pressure', 'replace'):{'low pressure':(np.random.normal, (20,)),
                                          'leak':        (np.random.normal, (25,))},
             ('leak', 'replace'):        {'leak':        (np.random.normal, (20,)),
                                          'low pressure':(np.random.normal, (20,))}}

problem_solution = hydraulic_reliability[('low pressure', 'replace')]
problem_solution['leak']

(<function RandomState.normal>, (25,))

In [9]:
dist, params = hydraulic_reliability['low pressure', 'tighten']

In [11]:
dist(params)

array([8.44542028])

In [54]:
hydraulic.problem_solution_pairs[('leak', 'tighten')].get_ttf()

10.37867770459179

In [ ]:
x = np.arange()